In [1]:
pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## Load the pretrained BERT

In [2]:
from transformers import BertTokenizer, BertModel

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load pre-trained BERT model
model = BertModel.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
# Example sentences
sentence_1 = "I love playing football."
sentence_2 = "It is a great sport."

# Tokenizing with special tokens [CLS] and [SEP]
tokens = tokenizer(sentence_1, sentence_2, padding=True, truncation=True, return_tensors="pt")

print(tokens)


{'input_ids': tensor([[ 101, 1045, 2293, 2652, 2374, 1012,  102, 2009, 2003, 1037, 2307, 4368,
         1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
# Get embeddings from BERT
outputs = model(**tokens)

# Extract hidden states (last layer output)
context_vectors = outputs.last_hidden_state

print(context_vectors.shape)  # (batch_size, sequence_length, hidden_dim)


torch.Size([1, 14, 768])


In [5]:
from transformers import BertForSequenceClassification
import torch

# Load BERT with a classification head (2 classes: Positive/Negative)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Run BERT on input text
outputs = model(**tokens)

# Apply Softmax to get probabilities
probs = torch.softmax(outputs.logits, dim=-1)
print("Sentiment Probabilities:", probs)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment Probabilities: tensor([[0.5350, 0.4650]], grad_fn=<SoftmaxBackward0>)


In [6]:
from transformers import BertForQuestionAnswering

# Load BERT for QA
qa_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Question & Context
question = "What is the capital of France?"
context = "Paris is the capital of France."

# Tokenizing with [CLS] and [SEP]
qa_tokens = tokenizer(question, context, return_tensors="pt")

# Get Answer Start & End Scores
outputs = qa_model(**qa_tokens)
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

# Extract the answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(qa_tokens["input_ids"][0][answer_start:answer_end]))
print("Predicted Answer:", answer)


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicted Answer: paris


✅ We used BERT for tokenization, classification, and question answering.

✅ [CLS] token is used for classification (Sentiment Analysis).

✅ [SEP] token separates questions and answers (QA Task).

✅ BERT outputs context-aware word embeddings for each token.